In [41]:
import json
from pathlib import Path
import pandas as pd
from collections import Counter
import requests
import os
from tqdm import tqdm

In [51]:
data_base = Path("data")

In [22]:
data = json.loads(Path("data/campusplan.json").read_text())
data[0]

{'id': 1,
 'title': 'Egon-Eiermann-Hörsaal (20.40)',
 'category': 'lecturehall',
 'aliasList': [],
 'positionList': [['49.01097', '8.41095']],
 'url': None}

In [29]:
df = pd.DataFrame(data)
len(df)

821

## Some entries have 2 positions

In [30]:
Counter(df.positionList.apply(len))

Counter({1: 814, 2: 7})

In [31]:
df[df.positionList.apply(len) == 2]

,id,title,category,aliasList,positionList,url
544,554,"Thermische Strömungsmaschinen, Institut für",institute,[],"[[49.01253, 8.41661], [49.00990, 8.41322]]",http://www.its.kit.edu
566,581,"Technikzukünfte - Philosophie, Institut für",institute,[],"[[49.00948, 8.39325], [49.01257, 8.40664]]",http://www.philosophie.kit.edu/
690,745,Zentrum für Mediales Lernen (ZML),facility,[],"[[49.00751, 8.40425], [49.01136, 8.41887]]",http://www.zml.kit.edu/
694,750,MINT-Kolleg,facility,[],"[[49.01126, 8.40903], [49.01035, 8.41879]]",https://www.mint-kolleg.kit.edu/
709,769,Fachschaft WiWi,institute,[],"[[49.00918, 8.40966], [49.01265, 8.40663]]",http://www.fachschaft.org/
771,842,40.50,building,[],"[[49.01418, 8.41438], [49.01418, 8.41438]]",None
772,843,40.51,building,[],"[[49.01414, 8.41491], [49.01414, 8.41491]]",None


We deal with this (in the pandas world) by exploding based on the positionList. We do this because in the following data enrichment, we want to have a unique position (lat and long) per row.

Later, we perhaps want a normalized SQL schema.

In [32]:
df = df.explode("positionList")
len(df)

828

In [34]:
df.head()

,id,title,category,aliasList,positionList,url
0,1,Egon-Eiermann-Hörsaal (20.40),lecturehall,[],"[49.01097, 8.41095]",None
1,2,Sport-Hörsaal (40.40),lecturehall,[],"[49.01580, 8.41636]",None
2,3,Hochspannungstechnik-Hörsaal (30.35),lecturehall,[],"[49.01197, 8.41272]",None
3,4,Johann-Gottfried-Tulla-Hörsaal (11.40),lecturehall,[],"[49.01023, 8.41187]",None
4,5,Hörsaal am Fasanengarten (50.35),lecturehall,[],"[49.01472, 8.42042]",None


In [52]:
df.to_csv(data_base / "campusplan.csv", index=False)

## Data enrichment

In [37]:
df.loc[0]

id                                          1
title           Egon-Eiermann-Hörsaal (20.40)
category                          lecturehall
aliasList                                  []
positionList              [49.01097, 8.41095]
url                                      None
Name: 0, dtype: object

In [60]:
def reverse_geocode(lat, lon):
    url = f"https://nominatim.openstreetmap.org/reverse?format=geojson&lat={lat}&lon={lon}"
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    return response.json()

def get_details(osm_type, osm_id):
    # Convert osm_type to API-required short form
    osm_type_map = {'node': 'N', 'way': 'W', 'relation': 'R'}
    type_code = osm_type_map[osm_type.lower()]
    url = f"https://nominatim.openstreetmap.org/details?osmtype={type_code}&osmid={osm_id}&format=json"
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    return response.json()

def enrich_and_save(df, output_dir: Path, refetch: bool = False):
    output_dir.mkdir(parents=True, exist_ok=refetch)
    for index, row in tqdm(df.iterrows(), total=len(df)):
        lat, lon = row["positionList"]
        rev_data = reverse_geocode(lat, lon)

        # Extract osm_type and osm_id
        try:
            osm_type = rev_data["features"][0]["properties"]["osm_type"]
            osm_id = rev_data["features"][0]["properties"]["osm_id"]
        except (KeyError, IndexError) as e:
            tqdm.write(f"Problem for {i}: {e}")

        details_data = get_details(osm_type, osm_id)

        # Save raw data
        with open(os.path.join(output_dir, f"{row['id']}_reverse.json"), "w") as f:
            json.dump(rev_data, f, indent=2)
        with open(os.path.join(output_dir, f"{row['id']}_details.json"), "w") as f:
            json.dump(details_data, f, indent=2)

In [64]:
df.iloc[-1:]

,id,title,category,aliasList,positionList,url
820,896,214,building,[],"[49.09085, 8.43021]",None


In [66]:
enrich_and_save(df, data_base / "nominatim_cache", refetch = True)

100%|████████████████████████████████████████████████████████████████████████████| 828/828 [1:31:21<00:00,  6.62s/it]


## Transform / merge

Explore the data in `explore_data.ipynb`. Adjust transformation based on insights.

- `rev_` prefixed columns come from data from reverse geocoding.
- `det_` prefixed columns come from data from OSM details.

In [86]:
def get_cached_details(id: str):
    p = data_base / "nominatim_cache"
    return json.loads((p / f"{id}_details.json").read_text())

def get_cached_reverse(id: str):
    p = data_base / "nominatim_cache"
    return json.loads((p / f"{id}_reverse.json").read_text())
    

In [89]:
df["reverse"] = df.id.apply(get_cached_reverse)
df["details"] = df.id.apply(get_cached_details)

In [90]:
df.head()

,id,title,category,aliasList,positionList,url,reverse,details
0,1,Egon-Eiermann-Hörsaal (20.40),lecturehall,[],"[49.01097, 8.41095]",None,"{'type': 'FeatureCollection', 'licence': 'Data...","{'place_id': 113347755, 'parent_place_id': 113..."
1,2,Sport-Hörsaal (40.40),lecturehall,[],"[49.01580, 8.41636]",None,"{'type': 'FeatureCollection', 'licence': 'Data...","{'place_id': 112652593, 'parent_place_id': 112..."
2,3,Hochspannungstechnik-Hörsaal (30.35),lecturehall,[],"[49.01197, 8.41272]",None,"{'type': 'FeatureCollection', 'licence': 'Data...","{'place_id': 112652149, 'parent_place_id': 112..."
3,4,Johann-Gottfried-Tulla-Hörsaal (11.40),lecturehall,[],"[49.01023, 8.41187]",None,"{'type': 'FeatureCollection', 'licence': 'Data...","{'place_id': 112652768, 'parent_place_id': 113..."
4,5,Hörsaal am Fasanengarten (50.35),lecturehall,[],"[49.01472, 8.42042]",None,"{'type': 'FeatureCollection', 'licence': 'Data...","{'place_id': 113642624, 'parent_place_id': 112..."


### from reverse mapping
`["features"][0]["properties"][[type, category, name, display_name]]`

In [91]:
for property_name in ("type", "category", "name", "display_name"):
    df[f"rev_{property_name}"] = df["reverse"].apply(lambda x: x["features"][0]["properties"][property_name])

In [92]:
df.columns

Index(['id', 'title', 'category', 'aliasList', 'positionList', 'url',
       'reverse', 'details', 'rev_type', 'rev_category', 'rev_name',
       'rev_display_name'],
      dtype='object')

### from details
`["extratags"]["wheelchair(:description),opening hours,contact:phone"]`

In [95]:
for extra_tag_name in ("opening_hours", "contact:phone", "wheelchair", "wheelchair:description"):
    df[f"det_{extra_tag_name}"] = df["details"].apply(lambda x: x["extratags"].get(extra_tag_name))

In [96]:
df.columns

Index(['id', 'title', 'category', 'aliasList', 'positionList', 'url',
       'reverse', 'details', 'rev_type', 'rev_category', 'rev_name',
       'rev_display_name', 'det_opening_hours', 'det_contact:phone',
       'det_wheelchair', 'det_wheelchair:description'],
      dtype='object')

In [102]:
del df["reverse"]
del df["details"]

df.to_csv(data_base / "campusplan_enriched.csv", index=False)